In [722]:
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TUSHAR\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Here i have approached the problem using a Content Based Recommendation What i have done is that I have used preview Content of Blogs and Tags as the attributes to classify and recommend the Title for the Users The other aproach is the one which I have Mentioned at the last of this File

As the tags Forms basis for Recommending the users the users having most number of preffered tags as per their previous vists are recommended accordingly this reduces the effort for settting the filter on the website for search 


# >The First Process is the Text processing and filtering
# >Further the most number of tags and words are encountered in the data
# >Then I fit the train data in Tfidf having ngram from (1,5) this can be changed accordingly to have a more detailed 
# and robust model
# >Further the label model is encoded using MutliLabelBinarizer
# >Then the model is train on traing data set In One Vs All classifier for Multiclassification with Logistic
# Regression as model to train to have compressed
# >And then Prediction of Labels are done using Content which can act as feed to another model nad recommend the top 
# contents


In [579]:
df1=pd.read_csv("EXAMPREP.csv")

In [580]:
df1["Generated_tags"]=df1["Generated_tags"].astype(str)

In [704]:
df1["prev_cont"]=df1["prev_cont"].astype(str)

In [582]:
from sklearn.model_selection import train_test_split

In [583]:
df1.columns

Index(['title', 'content', 'url', 'prev_cont', 'prev_image', 'date',
       'author_name', 'author_link', 'tags', 'comments_cont', 'comments_date',
       'comments_id', 'Generated_tags'],
      dtype='object')

In [661]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_filter(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join([word for word in text.split() if word not in STOPWORDS]) # delete stopwors from text
    text = text.strip()
    return text

In [662]:
X_train, X_test,y_train,y_test = train_test_split(df1['prev_cont'].values,df1['Generated_tags'].values,test_size=0.2,random_state=10)

In [663]:
X_train=[text_filter(x) for x in X_train]

In [664]:
X_test=[text_filter(x) for x in X_test]

In [665]:
for i in range(len(y_train)):
    y_train[i]=y_train[i].split(",")

In [666]:
for i in range(len(X_test)):
    y_test[i]=y_test[i].split(",")

'form'

In [604]:
tags_counts={}
words_counts={}

from collections import Counter
z=dict()
for i in range(len(y_train)):
    for j in range(1):
        if y_train[i][j] in z.keys():
            z[y_train[i][j]] +=1
        else:
            z[y_train[i][j]] = 1
tags_counts=z            
            
p=dict()
for i in range(len(X_train)):
        for s in X_train[i].split():
            if s in p.keys():
                p[s] +=1
            else:
                p[s] = 1
words_counts=p   

In [606]:
from collections import Counter
tags_counts = Counter([item for taglist in y_train for item in taglist])
words_counts = Counter([word for line in X_train for word in line.split(' ')])

In [608]:
most_common_tags = sorted(tags_counts.items(), key=lambda x: x[1], reverse=True)[:3]
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:3]



In [609]:
most_common_words

[('2020', 549), ('check', 155), ('exam', 111)]

In [617]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [618]:
def tfidf_features(X_train,X_test):
    
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,5), max_df=0.9, min_df=5, token_pattern='(\S+)')
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    return X_train_tfidf

In [628]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,5), max_df=0.9, min_df=5, token_pattern='(\S+)')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [632]:
from sklearn.neighbors import NearestNeighbors

In [646]:
from sklearn.preprocessing import MultiLabelBinarizer

In [647]:
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
y_train = mlb.fit_transform(y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:935: UserWarning: unknown class(es) [0, 1] will be ignored
  .format(sorted(unknown, key=str)))


In [622]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier

In [623]:
def train_classifier(X_train, y_train):
    
    model = OneVsRestClassifier(LogisticRegression(penalty='l2', C=1.0))
    model.fit(X_train, y_train)
    return model

In [624]:
classifier_mybag = train_classifier(X_train_mybag, y_train)
classifier_tfidf = train_classifier(X_train_tfidf, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [657]:
y_test_predicted_labels_tfidf = classifier_tfidf.predict(X_test_tfidf)


In [651]:
Predict1=mlb.inverse_transform(y_test_predicted_labels_tfidf)
Predict2=mlb.inverse_transform(y_test_predicted_labels_tfidf2)

In [721]:
for i in range(100):
    print('prev cont:\t{}\nPredicted labels:\t{}'.format(
        X_test[i],
        ','.join(Predict1[i])))

prev cont:	new feature allows facebook users opt facial recognition social network
Predicted labels:	at,c,d,e,n,r
prev cont:	simple threedimensional structure reduces error outcompetes complex models doubles savings
Predicted labels:	at,c,d,e,io,n,r
prev cont:	gate
Predicted labels:	at,c,d,e,n,r
prev cont:	mat 2020 aima released application form mat 2020 december 27 2020 know details mat 2020 application form dates admit card result cutoff selection procedure
Predicted labels:	ai,application,at,c,cat,d,date,e,form,io,n,r
prev cont:	cat exam tough beast slay cracking exam half battle real test lies ahead cracking watpi rounds dream business schools youre competing creamy layer indias top talent competition going become immensely tough
Predicted labels:	at,c,cat,d,e,io,n,r
prev cont:	aihmct wat 2020 army institute hotel management catering technology start aihmct wat online registration anytime soon interested eligible candidates apply january 29 tentative
Predicted labels:	at,c,d,e,io,n

In [685]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

# Thus from tags we can determine the Title to recommend as I have used cosine similarity for distance calculation in tfidf we get almost closest value Further as the data can be more detailed with having a attribute of similarity between the blogs using tags which can work for Collaberative filtering as per the user based although this model uses Machile Learning models A one without one using simple nlp can be created The futher one with a robust structure can be created using SVD I have tried a basic version of Knn for this dataset although the filterings was not so good so i go up with this model